# Part I

##### Authors: Taniya Cai, Yimin You

## Introduction


* 1, estimate the average song duration in the playlist
* 2, a continuous analysis (parameter of interest is the mean song duration) and a binary analysis (parameter of interest is the proportion of songs longer than 180 seconds).
* 3, first: will be a Simple Random Sample, second: Stratified Sample stratified by song duration.



## Population Parameters

* 1, decide required sample size(N)
* 2, sample size for continuous population
* 3, sample size for binary population


## SRS

assumption, continuous population (sample mean, SE, the 95% confidence interval)
binary population (sample mean, SE, the 95% confidence interval)


## Stratified Sampling

In comparison with a simple random sampling, we can divide the population into sub-populations, then apply random sampling of each subgroups to more accurately reflect the population studied based on a set of chosen characteristics.

* Assumptions:

> All observations must be exactly under one and only one stratum.

> Sampling from each stratum must be random\n",

* Hypothesis:

> The value of the parameters chosen are the same across all stratum, that is , 1.) the proportion of personal income that exceeds $50K/yr is the same across all stratum, and 2.) the ........ is the same across all stratum.


## Analysis and Comparison

In [2]:
library(tidyverse)
library(GGally)
adult <- read.csv("adult.data", header = FALSE)
column_names <- c("age",
                  "workclass",
                  "final_weight",
                  "education", 
                  "education_num",
                  "marital_status",
                  "occupation",
                  "relationship",
                  "race",
                  "sex",
                  "capital_gain", 
                  "capital_loss", 
                  "hours_per_week", 
                  "native_country",
                  "income")
adult <- setNames(adult, column_names)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.3     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2



In [3]:
head(adult)

,age,workclass,final_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
,<int>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>
1,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
2,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
3,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
4,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
5,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
6,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [4]:
level <- as.character(levels(as.factor(adult$marital_status)))
level
status <- as.character(adult$marital_status)
for (i in 1:length(status)) {
    ifelse (status[i] == level[2] | status[i] == level[3] | status[i] == level[4], 
            status[i] <- " Married", 
            status[i] <- status[i])
}

[1] " Divorced"              " Married-AF-spouse"     " Married-civ-spouse"   
[4] " Married-spouse-absent" " Never-married"         " Separated"            
[7] " Widowed"

In [5]:
n = 2000 ## we set sample size to be 1000
N = nrow(adult) ## population size
## Get pproportional stratum sizes
adult$status <- as.factor(status)

marital_ns <- adult %>%
    group_by(status) %>%
    summarise(nn = n(), prop = n()/N, var = var(age), sd = sd(age)) %>%
    mutate(nc = nn*sd) %>%
    mutate(nprop = nc/sum(nc)) %>%
    mutate(nh = round(nprop*n))
marital_ns

status,nn,prop,var,sd,nc,nprop,nh
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Divorced,4443,0.13645158,108.1682,10.40040,46208.96,0.12740224,255
Married,15417,0.47348054,145.8976,12.07881,186218.98,0.51342242,1027
Never-married,10683,0.32809189,100.1449,10.00724,106907.35,0.29475315,590
Separated,1025,0.03147938,117.4184,10.83598,11106.88,0.03062265,61
Widowed,993,0.03049661,152.4127,12.34555,12259.14,0.03379954,68


since nn for Married-AF-spouse, Married-civ-spouse and Married-spouse-absent are too small, so we combine them into Married status.

In [6]:
## Stratums
marital_levels <- levels(adult$status)
nn <- marital_ns$nn
nvar <- marital_ns$var
nh <- marital_ns$nh
marital_levels

Divorced_sample <- adult %>%
    filter(status == marital_levels[1]) %>%
    pull(age) %>%
    sample(nh[1])

Married_sample <- adult %>%
    filter(status == marital_levels[2]) %>%
    pull(age) %>%
    sample(nh[2])

NeverMarried_sample <- adult %>%
    filter(status == marital_levels[3]) %>%
    pull(age) %>%
    sample(nh[3])

Separated_sample <- adult %>%
    filter(status == marital_levels[4]) %>%
    pull(age) %>%
    sample(nh[4])

Widowed_sample <- adult %>%
    filter(status == marital_levels[5]) %>%
    pull(age) %>%
    sample(nh[5])

[1] " Divorced"      " Married"       " Never-married" " Separated"    
[5] " Widowed"

* ##### Formula<br>
The estimate for the mean age is $\bar{y}_{str} = \sum_{h=1}^{H} ((\frac{N_{h}}{N})\bar{y}_{h}) $, when $N_{h}$ is the size of the $h^{th}$ strata, $N$ is the population size, and $\bar{y}_{h}$ is the mean age of $h^{th}$ strata. the standard error for this estimate is $SE_{\bar{y}_{str}} =  \sqrt{\sum_{h=1}^{H}((\frac{N_{h}}{N})^2SE_{\bar{y}_h}}$, where $SE_{\bar{y}_h} = \sqrt{(1-\frac{n}{N})*\frac{S^2_{h}}{n}}$ with $S_{h}$ being the standard deviation of the $h^{th}$ strata. The 95% confidence interval for $\bar{y}_{str}$ is $\bar{y}_{s}\pm 1.96*SE_{\bar{y}_{str}}$

In [7]:
## Estmitated mean age and SE
Divorced_est <- (nh[1]/n)*mean(Divorced_sample)
Married_est <- (nh[2]/n)*mean(Married_sample)
NeverMarried_est <- (nh[3]/n)*mean(NeverMarried_sample)
Separated_est <- (nh[4]/n)*mean(Separated_sample)
Widowed_est <- (nh[5]/n)*mean(Widowed_sample)

ybarhat <- sum(c(Divorced_est,Married_est,NeverMarried_est,Separated_est,Widowed_est))

## estimate SE
Divorced_se <- ((nn[1]/N)^2) * (1-(nh[1]/n))*(nvar[1]/nh[1])
Married_se <- ((nn[2]/N)^2) * (1-(nh[2]/n))*(nvar[2]/nh[2])
NeverMarried_se <- ((nn[3]/N)^2) * (1-(nh[3]/n))*(nvar[3]/nh[3])
Separated_se <- ((nn[4]/N)^2) * (1-(nh[4]/n))*(nvar[4]/nh[4])
Widowed_se <- ((nn[5]/N)^2) * (1-(nh[5]/n))*(nvar[5]/nh[5])

yhatse <- sum(c(Divorced_se,Married_se,NeverMarried_se,Separated_se,Widowed_se))

result_data <- data.frame(Estimate = ybarhat, SE = yhatse, true_parameter = mean(adult$age))
result_data

Estimate,SE,true_parameter
<dbl>,<dbl>,<dbl>
39.359,0.03912924,38.58165


In [13]:
phat.str.me <-1.96*yhatse
phat.str.confint <-c(ybarhat-phat.str.me,ybarhat+phat.str.me)
str.matrix <-matrix(c("SampleMean.","SE","LowerBound95%CI",
"Upper Bound95%CI",
round(ybarhat,3), round(yhatse,3),
round(phat.str.confint[1],3),
round(phat.str.confint[2],3)), nrow=4)
knitr::kable(x =str.matrix,col.names=c("","Value"))



|                 |Value  |
|:----------------|:------|
|SampleMean.      |39.359 |
|SE               |0.039  |
|LowerBound95%CI  |39.282 |
|Upper Bound95%CI |39.436 |

* ###### RESULT<br>
From our result, the estimate for mean age is 39.359. The standard error for this estimate is 0.03912924, Since our sample size sufficiently large (bigger than 25), by the central limit theorem, the distribution of our sample mean will be approximately normally distributed among many sample means. Therefore, we can conclude that we are 95% confident that the true mean age is between 39.282 and 39.436.

## Discussion 

## Conclusion 

# Part II